In [ ]:
def fib(n):
    """
    Assume n is an int >= 0
    Return Fibonacci of n
    """
    if n == 0 | n == 1:
        return 1
    else:
        return fib(n-1) + fib(n-2)

Addressing the inefficiency of the Fibonacci implementation above:

* Memoization solves the original problem top-down. It starts from the original problem, beaks it into subproblems, breaks the subproblems into subproblems. Each time it needs to solve a subproblem, it first tries to look up the answer in a table.
* Tabular is a bottom-up method, starts from the smallest problems, and stores the answers to those in a table. It then combines the solutions to these problems to solve the next smallest problems.

In [1]:
def fib_memo(n, memo = None):
    """
    :param n: an int >= 0
    :param memo:
    :return: Fibonacci of n
    """
    if memo == None:
        memo = {}
    if n == 0 or n == 1:
        return 1
    try:
        return memo[n]
    except KeyError:
        result = fib_memo(n-1, memo) + fib_memo(n-2, memo)
        memo[n] = result
        return result

In [15]:
def fib_tab(n):
    """
    :param n: an int >= 0
    :return: Fibonacci of n
    """
    tab = [1]*(n+1) #only first two values matter
    print('tab ', tab)
    for i in range(2, n+1):
        tab[i] = tab[i-1] + tab[i-2]
        print('tab[{}] = tab[{}-1] + tab[{}-2]'.format(i, i, i))
        print('{} = {} + {}'.format(tab[i], tab[i-1], tab[i-2]))
    return tab[n]

In [7]:
fib_memo(120)

8670007398507948658051921

In [16]:
fib_tab(5)

tab  [1, 1, 1, 1, 1, 1]
tab[2] = tab[2-1] + tab[2-2]
2 = 1 + 1
tab[3] = tab[3-1] + tab[3-2]
3 = 2 + 1
tab[4] = tab[4-1] + tab[4-2]
5 = 3 + 2
tab[5] = tab[5-1] + tab[5-2]
8 = 5 + 3


8

In [8]:
fib_tab(120)

8670007398507948658051921

Same asymptotics: both memo (top-down) and tab (bottom-up) do
𝑂
(
𝑛
)
O(n) work and
𝑂
(
𝑛
)
O(n) space if you store all results.

Constant-factor wins: bottom-up avoids recursion and dict lookups.

Top-down memo does many dict hits and pays Python call overhead each recursion.

Bottom-up uses straight-line loops and list (or variable) indexing—cheaper ops in Python.

Recursion limit: top-down recurses to depth
𝑛
n; for
𝑛
≳
900
n≳900 you risk hitting Python’s recursion limit. Bottom-up has no such risk.

When top-down can win: in problems where you don’t need all subproblems (sparse DAG), memo only computes what’s required. But Fibonacci uses every subproblem, so tabulation shines.

In [31]:
def make_change(coin_vals, change):
    """
    :param coin_vals: a list of ints
    :param change: an int >= 0
    :return: minimum number of coins needed to make change
    """
    # initialize with "infinity" for all values > 0
    dp = [float('inf')] * (change + 1)
    dp[0] = 0

    for amt in range(1, change + 1):
        for coin in coin_vals:
            print('coin = {}'.format(coin))
            print('amt = {}'.format(amt))
            print('coin <= amt = ', coin <= amt)
            if coin <= amt:
                dp[amt] = min(dp[amt], 1 + dp[amt - coin])
                print('dp[{}] = min(dp[{}], 1 + dp[{} - {}])'.format(amt,amt,amt,coin))
                print('min({}, 1 + {})'.format(dp[amt], dp[amt - coin]))
                print(dp)

    return dp[change]


In [32]:
make_change([1,5,8], 11)

coin = 1
amt = 1
coin <= amt =  True
dp[1] = min(dp[1], 1 + dp[1 - 1])
min(1, 1 + 0)
[0, 1, inf, inf, inf, inf, inf, inf, inf, inf, inf, inf]
coin = 5
amt = 1
coin <= amt =  False
coin = 8
amt = 1
coin <= amt =  False
coin = 1
amt = 2
coin <= amt =  True
dp[2] = min(dp[2], 1 + dp[2 - 1])
min(2, 1 + 1)
[0, 1, 2, inf, inf, inf, inf, inf, inf, inf, inf, inf]
coin = 5
amt = 2
coin <= amt =  False
coin = 8
amt = 2
coin <= amt =  False
coin = 1
amt = 3
coin <= amt =  True
dp[3] = min(dp[3], 1 + dp[3 - 1])
min(3, 1 + 2)
[0, 1, 2, 3, inf, inf, inf, inf, inf, inf, inf, inf]
coin = 5
amt = 3
coin <= amt =  False
coin = 8
amt = 3
coin <= amt =  False
coin = 1
amt = 4
coin <= amt =  True
dp[4] = min(dp[4], 1 + dp[4 - 1])
min(4, 1 + 3)
[0, 1, 2, 3, 4, inf, inf, inf, inf, inf, inf, inf]
coin = 5
amt = 4
coin <= amt =  False
coin = 8
amt = 4
coin <= amt =  False
coin = 1
amt = 5
coin <= amt =  True
dp[5] = min(dp[5], 1 + dp[5 - 1])
min(5, 1 + 4)
[0, 1, 2, 3, 4, 5, inf, inf, inf, inf, inf, inf]
coin =

3

So after the algorithm finishes looping through all amt from 1 to change,
the table dp is completely filled with the best solution for every smaller amount.

Example:

| i (amount) | dp[i] | Meaning           |
| ---------- | ----- | ----------------- |
| 0          | 0     | 0 coins to make 0 |
| 1          | 1     | 1 coin (1)        |
| 2          | 2     | 2 coins (1+1)     |
| 3          | 3     | 3 coins (1+1+1)   |
| 4          | 4     | 4 coins (1x4)     |
| 5          | 1     | 1 coin (5)        |
| …          | …     | …                 |
| 11         | 3     | 3 coins (5+5+1)   |


When you reach amt = change, you’ve solved the final goal —
the minimum coins to make exactly that total.

So dp[change] is the final answer.

In [36]:
def max_val(to_consider, avail):
    """
    :param to_consider: a list of items
    :param avail: a weight
    :return: a tuple the total value of a solution to the 0/1 knapsack problem and the items of that solution
    """
    if to_consider == [] or avail == 0:
        result = (0, ())
    elif to_consider[0].get_weight() > avail:
        #Explore right branch only
        result = max_val(to_consider[1:], avail)
    else:
        next_item = to_consider[0]
        #Explore left branch
        with_val, with_to_take = max_val(
            to_consider[1:],
            avail - next_item.get_weight()
        )
        with_val += next_item.get_value()
        #Explore right branch
        without_val, without_to_take = max_val(
            to_consider[1:],
            avail
        )
        #choose better branch
        if with_val > without_val:
            result = (with_val, with_to_take + (next_item,))
        else:
            result = (without_val, without_to_take)
    return result

Search Tree Implementation
* The tree is built top down starting with the root
* The first element is selected from the still to be considered items
    * If there is room for that item in the knapsack, a node is constructed that reflects the consequence of choosing to tak that item. By convention, we draw that as the left child
    * We also explore the consequence of not taking that item. This is the right child
* The process is then applied recursively to non-leaf children
* Finally, chose a node with the highest value that mees constraints
Computational Complexity
- Time based on number of nodes generated
- Number of levels is number of items to choose from Number of nodes at level _i_ is _2^i_
- So, if there are n items the number of node is
    $$\sum_{i=n}^{i=0} 2^i$$
    i.e. $$O(2^i+1)$$
* An obvious optimization: don't explore parts of tree that violate constraint (e.g., too many calories)
    * Doesn't change complexity
* Does this mean that brute force is never useful?


In [43]:
from mylib import build_many_items
def big_test(num_items, avail_wight):
    items = build_many_items(num_items, 10, 10)
    val, taken = max_val(items, avail_wight)
    print('Items Taken')
    for item in taken:
        print(item)
    print('Total value of items  taken =', val)

In [38]:
big_test(10, 40)

Items Taken

              8: <value:   4, weight:   1>

              6: <value:   8, weight:   4>

              3: <value:  10, weight:  10>

              2: <value:   7, weight:  10>

              1: <value:   9, weight:   7>

              0: <value:   9, weight:   7>
Total value of items  taken = 47


In [39]:
def fast_max_val(to_consider, avail, memo = {}):
    """"
    :param to_consider: a list of items
    :param avail: a weight
    :param memo: a memo
    :return: a tuple the total value of a solution to the 0/1 knapsack problem
    """
    if (len(to_consider), avail) in memo:
        result = memo[(len(to_consider), avail)]
    elif to_consider == [] or avail == 0:
        result = (0, ())
    elif to_consider[0].get_weight() > avail:
        #Explore right branch only
        result = fast_max_val(to_consider[1:], avail, memo)
    else:
        next_item = to_consider[0]
        #Explore left branch
        with_val, with_to_take = fast_max_val(
            to_consider[1:],
            avail - next_item.get_weight(),
            memo
        )
        with_val += next_item.get_value()
        #Explore right branch
        without_val, without_to_take = fast_max_val(
            to_consider[1:],
            avail,
            memo
        )

        #choose better branch
        if with_val > without_val:
            result = (with_val, with_to_take + (next_item,))
        else:
            result = (without_val, without_to_take)
    memo[(len(to_consider), avail)] = result
    return result

In [40]:
from mylib import build_many_items
def big_test(num_items, avail_wight):
    items = build_many_items(num_items, 10, 10)
    val, taken = fast_max_val(items, avail_wight)
    print('Items Taken')
    for item in taken:
        print(item)
    print('Total value of items  taken =', val)

In [42]:
big_test(40, 100)

Items Taken

              8: <value:   5, weight:   7>

              7: <value:   7, weight:   4>

              6: <value:   7, weight:   8>

              3: <value:   6, weight:   3>

              2: <value:  10, weight:   5>

             30: <value:   9, weight:   5>

             29: <value:   4, weight:   3>

             28: <value:  10, weight:   4>

             27: <value:   7, weight:   7>

             25: <value:  10, weight:   2>

             24: <value:   7, weight:   6>

             23: <value:   8, weight:   2>

             21: <value:   5, weight:   5>

             19: <value:   3, weight:   3>

             16: <value:   7, weight:   4>

             13: <value:   2, weight:   1>

             11: <value:   7, weight:   2>

             10: <value:  10, weight:   4>

              9: <value:   9, weight:   5>

              7: <value:  10, weight:   9>

              5: <value:  10, weight:   2>

              4: <value:   9, weight:   8>

              1: <v